<a href="https://colab.research.google.com/github/grabuffo/BrainStim_ANN_fMRI_HCP/blob/main/notebooks/HCP_ANN_Connectiviy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================
# 📦 Load preprocessed fMRI data + trained model (PyTorch ≥ 2.6 SAFE version)
# ============================================

# --- 1️⃣ Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- 2️⃣ Clone your GitHub repo (contains src/NPI.py) ---
!rm -rf /content/BrainStim_ANN_fMRI_HCP
!git clone https://github.com/grabuffo/BrainStim_ANN_fMRI_HCP.git

# --- 3️⃣ Define paths ---
import os, sys
repo_dir   = "/content/BrainStim_ANN_fMRI_HCP"
data_dir   = "/content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data"
preproc_dir = os.path.join(data_dir, "preprocessed_subjects")
models_dir  = os.path.join(preproc_dir, "trained_models_MLP")

# Add repo to Python path so src/NPI is visible
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# --- 4️⃣ Imports ---
import numpy as np
import torch
import gc
from src import NPI

print("✅ Imported NPI from:", os.path.join(repo_dir, "src"))
print("repo_dir:", repo_dir)
print("preproc_dir:", preproc_dir)
print("models_dir:", models_dir)

# --- 5️⃣ Choose subject ID ---
subj_id = "id_100206"   # 🔁 change to load another subject

# --- 6️⃣ Define file paths ---
signals_path = os.path.join(preproc_dir, f"{subj_id}_signals.npy")
inputs_path  = os.path.join(preproc_dir, f"{subj_id}_inputs.npy")
targets_path = os.path.join(preproc_dir, f"{subj_id}_targets.npy")
model_path   = os.path.join(models_dir,   f"{subj_id}_MLP.pt")  # or "_MLP_full.pt" if you used that name

# --- 7️⃣ Load preprocessed data ---
signals = np.load(signals_path)
inputs  = np.load(inputs_path)
targets = np.load(targets_path)

print(f"\n✅ Loaded preprocessed data for {subj_id}")
print(f"signals shape: {signals.shape}")
print(f"inputs  shape: {inputs.shape}")
print(f"targets shape: {targets.shape}")

# --- 8️⃣ Safely allowlist your model classes (needed since PyTorch ≥ 2.6) ---
import torch.serialization

# Find and allow all model classes defined inside NPI.py
import re
npi_file = os.path.join(repo_dir, "src", "NPI.py")
with open(npi_file, "r") as f:
    class_names = re.findall(r"class\s+([A-Za-z0-9_]+)\s*\(", f.read())

safe_classes = []
for name in class_names:
    try:
        cls = getattr(NPI, name)
        safe_classes.append(cls)
    except AttributeError:
        pass

if safe_classes:
    torch.serialization.add_safe_globals(safe_classes)
    print("✅ Allowed model classes:", [c.__name__ for c in safe_classes])
else:
    print("⚠️ No model classes detected in NPI.py")

# --- 9️⃣ Load trained model (FULL MODEL saved with torch.save(model, ...)) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path, map_location=device, weights_only=False)
model.eval()

print(f"\n✅ Loaded trained model from: {model_path}")
print(model)

# --- 🔟 Optional cleanup ---
gc.collect()
torch.cuda.empty_cache()
print("\n🎯 Model and data successfully loaded and ready for analysis.")


Mounted at /content/drive
Cloning into 'BrainStim_ANN_fMRI_HCP'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 122 (delta 34), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (122/122), 22.47 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (34/34), done.
✅ Imported NPI from: /content/BrainStim_ANN_fMRI_HCP/src
repo_dir: /content/BrainStim_ANN_fMRI_HCP
preproc_dir: /content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data/preprocessed_subjects
models_dir: /content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data/preprocessed_subjects/trained_models_MLP

✅ Loaded preprocessed data for id_100206
signals shape: (4680, 450)
inputs  shape: (4677, 1350)
targets shape: (4677, 450)
✅ Allowed model classes: ['ANN_MLP', 'ANN_CNN', 'ANN_RNN', 'ANN_VAR']

✅ Loaded trained model from: /content/drive/MyDrive/Colab Notebooks/Brain_Stim_ANN/data/preprocessed_subjects/trained

In [6]:
!ls /content/BrainStim_ANN_fMRI_HCP


notebooks  README.md  src
